Loading and preprocessing data...
X_train: (6096, 224, 224, 3), y_train: (6096,)
Number of classes: 8
Training model...


c:\Users\Sri_S\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 754ms/step - accuracy: 0.2770 - loss: 18.6957 - precision: 0.4518 - recall: 0.0321

153/153 ━━━━━━━━━━━━━━━━━━━━ 158s 891ms/step - accuracy: 0.2775 - loss: 18.6862 - precision: 0.4535 - recall: 0.0325 - val_accuracy: 0.1680 - val_loss: 15.1297 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 2/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 731ms/step - accuracy: 0.5031 - loss: 13.4568 - precision: 0.7231 - recall: 0.2950

153/153 ━━━━━━━━━━━━━━━━━━━━ 125s 814ms/step - accuracy: 0.5034 - loss: 13.4503 - precision: 0.7232 - recall: 0.2954 - val_accuracy: 0.5066 - val_loss: 10.9017 - val_precision: 0.7205 - val_recall: 0.1902 - learning_rate: 1.0000e-04
Epoch 3/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 694ms/step - accuracy: 0.6132 - loss: 9.8647 - precision: 0.7667 - recall: 0.4710

153/153 ━━━━━━━━━━━━━━━━━━━━ 118s 773ms/step - accuracy: 0.6133 - loss: 9.8602 - precision: 0.7668 - recall: 0.4711 - val_accuracy: 0.7434 - val_loss: 7.6329 - val_precision: 0.8101 - val_recall: 0.6574 - learning_rate: 1.0000e-04
Epoch 4/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 706ms/step - accuracy: 0.6640 - loss: 7.3369 - precision: 0.7846 - recall: 0.5471

153/153 ━━━━━━━━━━━━━━━━━━━━ 120s 786ms/step - accuracy: 0.6640 - loss: 7.3338 - precision: 0.7846 - recall: 0.5472 - val_accuracy: 0.7680 - val_loss: 5.7437 - val_precision: 0.8399 - val_recall: 0.6795 - learning_rate: 1.0000e-04
Epoch 5/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 706ms/step - accuracy: 0.7025 - loss: 5.5245 - precision: 0.7990 - recall: 0.6108

153/153 ━━━━━━━━━━━━━━━━━━━━ 120s 786ms/step - accuracy: 0.7025 - loss: 5.5223 - precision: 0.7989 - recall: 0.6108 - val_accuracy: 0.8025 - val_loss: 4.2897 - val_precision: 0.8574 - val_recall: 0.7393 - learning_rate: 1.0000e-04
Epoch 6/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 702ms/step - accuracy: 0.7281 - loss: 4.1883 - precision: 0.8181 - recall: 0.6452

153/153 ━━━━━━━━━━━━━━━━━━━━ 120s 784ms/step - accuracy: 0.7281 - loss: 4.1867 - precision: 0.8181 - recall: 0.6452 - val_accuracy: 0.8090 - val_loss: 3.2740 - val_precision: 0.8509 - val_recall: 0.7811 - learning_rate: 1.0000e-04
Epoch 7/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 704ms/step - accuracy: 0.7524 - loss: 3.2269 - precision: 0.8268 - recall: 0.6840

153/153 ━━━━━━━━━━━━━━━━━━━━ 120s 784ms/step - accuracy: 0.7524 - loss: 3.2258 - precision: 0.8268 - recall: 0.6840 - val_accuracy: 0.8377 - val_loss: 2.4942 - val_precision: 0.8758 - val_recall: 0.8033 - learning_rate: 1.0000e-04
Epoch 8/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 698ms/step - accuracy: 0.7584 - loss: 2.5485 - precision: 0.8322 - recall: 0.6959

153/153 ━━━━━━━━━━━━━━━━━━━━ 119s 778ms/step - accuracy: 0.7585 - loss: 2.5476 - precision: 0.8322 - recall: 0.6959 - val_accuracy: 0.8221 - val_loss: 2.0096 - val_precision: 0.8560 - val_recall: 0.7943 - learning_rate: 1.0000e-04
Epoch 9/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 691ms/step - accuracy: 0.7782 - loss: 2.0409 - precision: 0.8360 - recall: 0.7224

153/153 ━━━━━━━━━━━━━━━━━━━━ 119s 779ms/step - accuracy: 0.7782 - loss: 2.0403 - precision: 0.8360 - recall: 0.7224 - val_accuracy: 0.8180 - val_loss: 1.6663 - val_precision: 0.8470 - val_recall: 0.7984 - learning_rate: 1.0000e-04
Epoch 10/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 692ms/step - accuracy: 0.7900 - loss: 1.6646 - precision: 0.8413 - recall: 0.7338

153/153 ━━━━━━━━━━━━━━━━━━━━ 118s 771ms/step - accuracy: 0.7900 - loss: 1.6642 - precision: 0.8413 - recall: 0.7338 - val_accuracy: 0.8402 - val_loss: 1.3259 - val_precision: 0.8709 - val_recall: 0.8180 - learning_rate: 1.0000e-04
Epoch 11/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 693ms/step - accuracy: 0.7909 - loss: 1.4100 - precision: 0.8394 - recall: 0.7378

153/153 ━━━━━━━━━━━━━━━━━━━━ 119s 774ms/step - accuracy: 0.7909 - loss: 1.4096 - precision: 0.8394 - recall: 0.7378 - val_accuracy: 0.8393 - val_loss: 1.1431 - val_precision: 0.8620 - val_recall: 0.8246 - learning_rate: 1.0000e-04
Epoch 12/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 694ms/step - accuracy: 0.7892 - loss: 1.2389 - precision: 0.8368 - recall: 0.7375

153/153 ━━━━━━━━━━━━━━━━━━━━ 119s 774ms/step - accuracy: 0.7893 - loss: 1.2385 - precision: 0.8368 - recall: 0.7376 - val_accuracy: 0.8500 - val_loss: 0.9426 - val_precision: 0.8781 - val_recall: 0.8328 - learning_rate: 1.0000e-04
Epoch 13/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 702ms/step - accuracy: 0.8247 - loss: 1.0079 - precision: 0.8657 - recall: 0.7784

153/153 ━━━━━━━━━━━━━━━━━━━━ 120s 782ms/step - accuracy: 0.8246 - loss: 1.0079 - precision: 0.8656 - recall: 0.7783 - val_accuracy: 0.8451 - val_loss: 0.8472 - val_precision: 0.8701 - val_recall: 0.8287 - learning_rate: 1.0000e-04
Epoch 14/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 693ms/step - accuracy: 0.8208 - loss: 0.9284 - precision: 0.8565 - recall: 0.7743

153/153 ━━━━━━━━━━━━━━━━━━━━ 119s 774ms/step - accuracy: 0.8208 - loss: 0.9282 - precision: 0.8565 - recall: 0.7743 - val_accuracy: 0.8615 - val_loss: 0.7471 - val_precision: 0.8811 - val_recall: 0.8385 - learning_rate: 1.0000e-04
Epoch 15/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 700ms/step - accuracy: 0.8159 - loss: 0.8247 - precision: 0.8568 - recall: 0.7790

153/153 ━━━━━━━━━━━━━━━━━━━━ 119s 780ms/step - accuracy: 0.8160 - loss: 0.8246 - precision: 0.8568 - recall: 0.7790 - val_accuracy: 0.8467 - val_loss: 0.7395 - val_precision: 0.8679 - val_recall: 0.8238 - learning_rate: 1.0000e-04
Epoch 16/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 696ms/step - accuracy: 0.8346 - loss: 0.7420 - precision: 0.8674 - recall: 0.7998

153/153 ━━━━━━━━━━━━━━━━━━━━ 119s 775ms/step - accuracy: 0.8345 - loss: 0.7420 - precision: 0.8673 - recall: 0.7997 - val_accuracy: 0.8426 - val_loss: 0.6720 - val_precision: 0.8677 - val_recall: 0.8336 - learning_rate: 1.0000e-04
Epoch 17/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 698ms/step - accuracy: 0.8147 - loss: 0.7270 - precision: 0.8505 - recall: 0.7770

153/153 ━━━━━━━━━━━━━━━━━━━━ 119s 778ms/step - accuracy: 0.8148 - loss: 0.7268 - precision: 0.8506 - recall: 0.7770 - val_accuracy: 0.8541 - val_loss: 0.6082 - val_precision: 0.8746 - val_recall: 0.8344 - learning_rate: 1.0000e-04
Epoch 18/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 697ms/step - accuracy: 0.8371 - loss: 0.6490 - precision: 0.8699 - recall: 0.8057

153/153 ━━━━━━━━━━━━━━━━━━━━ 119s 776ms/step - accuracy: 0.8371 - loss: 0.6491 - precision: 0.8699 - recall: 0.8057 - val_accuracy: 0.8615 - val_loss: 0.5619 - val_precision: 0.8856 - val_recall: 0.8500 - learning_rate: 1.0000e-04
Epoch 19/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 119s 775ms/step - accuracy: 0.8401 - loss: 0.6329 - precision: 0.8676 - recall: 0.8058 - val_accuracy: 0.8484 - val_loss: 0.5707 - val_precision: 0.8656 - val_recall: 0.8287 - learning_rate: 1.0000e-04
Epoch 20/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 699ms/step - accuracy: 0.8439 - loss: 0.5862 - precision: 0.8749 - recall: 0.8139

153/153 ━━━━━━━━━━━━━━━━━━━━ 119s 779ms/step - accuracy: 0.8440 - loss: 0.5862 - precision: 0.8749 - recall: 0.8139 - val_accuracy: 0.8672 - val_loss: 0.5066 - val_precision: 0.8866 - val_recall: 0.8590 - learning_rate: 1.0000e-04
Epoch 21/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 119s 777ms/step - accuracy: 0.8559 - loss: 0.5620 - precision: 0.8818 - recall: 0.8270 - val_accuracy: 0.8623 - val_loss: 0.5116 - val_precision: 0.8826 - val_recall: 0.8500 - learning_rate: 1.0000e-04
Epoch 22/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 119s 778ms/step - accuracy: 0.8516 - loss: 0.5422 - precision: 0.8739 - recall: 0.8230 - val_accuracy: 0.8582 - val_loss: 0.5239 - val_precision: 0.8735 - val_recall: 0.8492 - learning_rate: 1.0000e-04
Epoch 23/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 703ms/step - accuracy: 0.8612 - loss: 0.5053 - precision: 0.8881 - recall: 0.8376

153/153 ━━━━━━━━━━━━━━━━━━━━ 120s 784ms/step - accuracy: 0.8612 - loss: 0.5053 - precision: 0.8881 - recall: 0.8376 - val_accuracy: 0.8689 - val_loss: 0.4894 - val_precision: 0.8808 - val_recall: 0.8598 - learning_rate: 1.0000e-04
Epoch 24/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 703ms/step - accuracy: 0.8552 - loss: 0.5110 - precision: 0.8852 - recall: 0.8353

153/153 ━━━━━━━━━━━━━━━━━━━━ 120s 784ms/step - accuracy: 0.8552 - loss: 0.5110 - precision: 0.8852 - recall: 0.8353 - val_accuracy: 0.8721 - val_loss: 0.4757 - val_precision: 0.8841 - val_recall: 0.8631 - learning_rate: 1.0000e-04
Epoch 25/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 120s 781ms/step - accuracy: 0.8634 - loss: 0.4762 - precision: 0.8874 - recall: 0.8422 - val_accuracy: 0.8541 - val_loss: 0.4936 - val_precision: 0.8737 - val_recall: 0.8451 - learning_rate: 1.0000e-04
Epoch 26/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 120s 783ms/step - accuracy: 0.8659 - loss: 0.4828 - precision: 0.8913 - recall: 0.8410 - val_accuracy: 0.8533 - val_loss: 0.4985 - val_precision: 0.8754 - val_recall: 0.8467 - learning_rate: 1.0000e-04
Epoch 27/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 120s 784ms/step - accuracy: 0.8634 - loss: 0.4765 - precision: 0.8828 - recall: 0.8431 - val_accuracy: 0.8549 - val_loss: 0.5188 - val_precision: 0.8749 - val_recall: 0.8369 - learning_rate: 1.0000e-04
Epoch 28/100
153/153 ━━━━━━━━━━━━━━━━

153/153 ━━━━━━━━━━━━━━━━━━━━ 120s 786ms/step - accuracy: 0.8771 - loss: 0.4327 - precision: 0.8949 - recall: 0.8560 - val_accuracy: 0.8787 - val_loss: 0.4259 - val_precision: 0.8939 - val_recall: 0.8697 - learning_rate: 5.0000e-05
Epoch 29/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 120s 780ms/step - accuracy: 0.8777 - loss: 0.4140 - precision: 0.8990 - recall: 0.8619 - val_accuracy: 0.8730 - val_loss: 0.4302 - val_precision: 0.8872 - val_recall: 0.8639 - learning_rate: 5.0000e-05
Epoch 30/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 120s 783ms/step - accuracy: 0.8812 - loss: 0.4092 - precision: 0.8994 - recall: 0.8630 - val_accuracy: 0.8721 - val_loss: 0.4343 - val_precision: 0.8921 - val_recall: 0.8672 - learning_rate: 5.0000e-05
Epoch 31/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 120s 781ms/step - accuracy: 0.8881 - loss: 0.3963 - precision: 0.9060 - recall: 0.8749 - val_accuracy: 0.8730 - val_loss: 0.4270 - val_precision: 0.8835 - val_recall: 0.8639 - learning_rate: 5.0000e-05
Epoch 32/100
153/153 ━━━━━━━━━━━━━━━━

153/153 ━━━━━━━━━━━━━━━━━━━━ 121s 792ms/step - accuracy: 0.8824 - loss: 0.3896 - precision: 0.9033 - recall: 0.8702 - val_accuracy: 0.8730 - val_loss: 0.4229 - val_precision: 0.8881 - val_recall: 0.8656 - learning_rate: 2.5000e-05
Epoch 33/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 703ms/step - accuracy: 0.8932 - loss: 0.3851 - precision: 0.9118 - recall: 0.8746

153/153 ━━━━━━━━━━━━━━━━━━━━ 120s 784ms/step - accuracy: 0.8932 - loss: 0.3850 - precision: 0.9118 - recall: 0.8746 - val_accuracy: 0.8762 - val_loss: 0.4137 - val_precision: 0.8875 - val_recall: 0.8730 - learning_rate: 2.5000e-05
Epoch 34/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 121s 788ms/step - accuracy: 0.8906 - loss: 0.3775 - precision: 0.9069 - recall: 0.8771 - val_accuracy: 0.8811 - val_loss: 0.4167 - val_precision: 0.8934 - val_recall: 0.8721 - learning_rate: 2.5000e-05
Epoch 35/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 716ms/step - accuracy: 0.9019 - loss: 0.3590 - precision: 0.9158 - recall: 0.8875

153/153 ━━━━━━━━━━━━━━━━━━━━ 122s 797ms/step - accuracy: 0.9018 - loss: 0.3591 - precision: 0.9157 - recall: 0.8874 - val_accuracy: 0.8738 - val_loss: 0.4103 - val_precision: 0.8893 - val_recall: 0.8689 - learning_rate: 2.5000e-05
Epoch 36/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 120s 783ms/step - accuracy: 0.8961 - loss: 0.3680 - precision: 0.9112 - recall: 0.8772 - val_accuracy: 0.8795 - val_loss: 0.4155 - val_precision: 0.8929 - val_recall: 0.8746 - learning_rate: 2.5000e-05
Epoch 37/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 719ms/step - accuracy: 0.8967 - loss: 0.3633 - precision: 0.9119 - recall: 0.8794

153/153 ━━━━━━━━━━━━━━━━━━━━ 123s 802ms/step - accuracy: 0.8966 - loss: 0.3633 - precision: 0.9119 - recall: 0.8793 - val_accuracy: 0.8877 - val_loss: 0.4022 - val_precision: 0.8982 - val_recall: 0.8828 - learning_rate: 2.5000e-05
Epoch 38/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 120s 786ms/step - accuracy: 0.8912 - loss: 0.3735 - precision: 0.9051 - recall: 0.8759 - val_accuracy: 0.8861 - val_loss: 0.4106 - val_precision: 0.8977 - val_recall: 0.8779 - learning_rate: 2.5000e-05
Epoch 39/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 712ms/step - accuracy: 0.9052 - loss: 0.3408 - precision: 0.9177 - recall: 0.8894

153/153 ━━━━━━━━━━━━━━━━━━━━ 121s 793ms/step - accuracy: 0.9052 - loss: 0.3409 - precision: 0.9177 - recall: 0.8894 - val_accuracy: 0.8836 - val_loss: 0.3997 - val_precision: 0.8965 - val_recall: 0.8803 - learning_rate: 2.5000e-05
Epoch 40/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 121s 791ms/step - accuracy: 0.8962 - loss: 0.3482 - precision: 0.9112 - recall: 0.8857 - val_accuracy: 0.8869 - val_loss: 0.3998 - val_precision: 0.8961 - val_recall: 0.8836 - learning_rate: 2.5000e-05
Epoch 41/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 706ms/step - accuracy: 0.9056 - loss: 0.3441 - precision: 0.9158 - recall: 0.8934

153/153 ━━━━━━━━━━━━━━━━━━━━ 120s 786ms/step - accuracy: 0.9056 - loss: 0.3442 - precision: 0.9158 - recall: 0.8934 - val_accuracy: 0.8877 - val_loss: 0.3947 - val_precision: 0.9041 - val_recall: 0.8811 - learning_rate: 2.5000e-05
Epoch 42/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 124s 812ms/step - accuracy: 0.8970 - loss: 0.3501 - precision: 0.9113 - recall: 0.8826 - val_accuracy: 0.8902 - val_loss: 0.3957 - val_precision: 0.9053 - val_recall: 0.8852 - learning_rate: 2.5000e-05
Epoch 43/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 122s 796ms/step - accuracy: 0.9015 - loss: 0.3477 - precision: 0.9150 - recall: 0.8903 - val_accuracy: 0.8902 - val_loss: 0.3988 - val_precision: 0.9031 - val_recall: 0.8861 - learning_rate: 2.5000e-05
Epoch 44/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 124s 807ms/step - accuracy: 0.9017 - loss: 0.3394 - precision: 0.9147 - recall: 0.8883 - val_accuracy: 0.8779 - val_loss: 0.4207 - val_precision: 0.8895 - val_recall: 0.8713 - learning_rate: 2.5000e-05
Epoch 45/100
153/153 ━━━━━━━━━━━━━━━━

153/153 ━━━━━━━━━━━━━━━━━━━━ 128s 833ms/step - accuracy: 0.8927 - loss: 0.3500 - precision: 0.9092 - recall: 0.8834 - val_accuracy: 0.8877 - val_loss: 0.3915 - val_precision: 0.8997 - val_recall: 0.8820 - learning_rate: 1.2500e-05
Epoch 46/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 122s 793ms/step - accuracy: 0.9033 - loss: 0.3271 - precision: 0.9150 - recall: 0.8923 - val_accuracy: 0.8861 - val_loss: 0.3939 - val_precision: 0.8997 - val_recall: 0.8828 - learning_rate: 1.2500e-05
Epoch 47/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 122s 797ms/step - accuracy: 0.9106 - loss: 0.3184 - precision: 0.9211 - recall: 0.8981 - val_accuracy: 0.8844 - val_loss: 0.3940 - val_precision: 0.8969 - val_recall: 0.8844 - learning_rate: 1.2500e-05
Epoch 48/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 726ms/step - accuracy: 0.9062 - loss: 0.3228 - precision: 0.9183 - recall: 0.8945

153/153 ━━━━━━━━━━━━━━━━━━━━ 124s 809ms/step - accuracy: 0.9062 - loss: 0.3228 - precision: 0.9183 - recall: 0.8945 - val_accuracy: 0.8803 - val_loss: 0.3893 - val_precision: 0.8923 - val_recall: 0.8762 - learning_rate: 1.2500e-05
Epoch 49/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9109 - loss: 0.3170 - precision: 0.9182 - recall: 0.8998

153/153 ━━━━━━━━━━━━━━━━━━━━ 180s 1s/step - accuracy: 0.9109 - loss: 0.3170 - precision: 0.9182 - recall: 0.8997 - val_accuracy: 0.8877 - val_loss: 0.3862 - val_precision: 0.9006 - val_recall: 0.8836 - learning_rate: 1.2500e-05
Epoch 50/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 127s 829ms/step - accuracy: 0.9155 - loss: 0.3026 - precision: 0.9274 - recall: 0.9069 - val_accuracy: 0.8844 - val_loss: 0.3868 - val_precision: 0.8974 - val_recall: 0.8820 - learning_rate: 1.2500e-05
Epoch 51/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 735ms/step - accuracy: 0.8966 - loss: 0.3439 - precision: 0.9092 - recall: 0.8835

153/153 ━━━━━━━━━━━━━━━━━━━━ 126s 823ms/step - accuracy: 0.8967 - loss: 0.3438 - precision: 0.9092 - recall: 0.8836 - val_accuracy: 0.8844 - val_loss: 0.3785 - val_precision: 0.9003 - val_recall: 0.8811 - learning_rate: 1.2500e-05
Epoch 52/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 2490s 16s/step - accuracy: 0.9059 - loss: 0.3148 - precision: 0.9165 - recall: 0.8970 - val_accuracy: 0.8779 - val_loss: 0.3978 - val_precision: 0.8897 - val_recall: 0.8730 - learning_rate: 1.2500e-05
Epoch 53/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 119s 779ms/step - accuracy: 0.9123 - loss: 0.3101 - precision: 0.9258 - recall: 0.8980 - val_accuracy: 0.8861 - val_loss: 0.3891 - val_precision: 0.8952 - val_recall: 0.8820 - learning_rate: 1.2500e-05
Epoch 54/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 133s 866ms/step - accuracy: 0.9012 - loss: 0.3288 - precision: 0.9113 - recall: 0.8884 - val_accuracy: 0.8852 - val_loss: 0.3827 - val_precision: 0.9003 - val_recall: 0.8811 - learning_rate: 1.2500e-05
Epoch 55/100
153/153 ━━━━━━━━━━━━━━━━━

153/153 ━━━━━━━━━━━━━━━━━━━━ 326s 2s/step - accuracy: 0.9125 - loss: 0.2994 - precision: 0.9245 - recall: 0.9041 - val_accuracy: 0.8836 - val_loss: 0.3783 - val_precision: 0.8958 - val_recall: 0.8811 - learning_rate: 1.5625e-06
Epoch 62/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 719ms/step - accuracy: 0.9113 - loss: 0.3074 - precision: 0.9233 - recall: 0.8991

153/153 ━━━━━━━━━━━━━━━━━━━━ 124s 808ms/step - accuracy: 0.9113 - loss: 0.3074 - precision: 0.9233 - recall: 0.8991 - val_accuracy: 0.8852 - val_loss: 0.3775 - val_precision: 0.8950 - val_recall: 0.8803 - learning_rate: 1.5625e-06
Epoch 63/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 767ms/step - accuracy: 0.9165 - loss: 0.3047 - precision: 0.9262 - recall: 0.9010

153/153 ━━━━━━━━━━━━━━━━━━━━ 132s 859ms/step - accuracy: 0.9164 - loss: 0.3048 - precision: 0.9262 - recall: 0.9010 - val_accuracy: 0.8844 - val_loss: 0.3752 - val_precision: 0.8974 - val_recall: 0.8820 - learning_rate: 1.5625e-06
Epoch 64/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 131s 855ms/step - accuracy: 0.9148 - loss: 0.3074 - precision: 0.9256 - recall: 0.9033 - val_accuracy: 0.8852 - val_loss: 0.3760 - val_precision: 0.8961 - val_recall: 0.8836 - learning_rate: 1.5625e-06
Epoch 65/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 131s 852ms/step - accuracy: 0.9122 - loss: 0.2942 - precision: 0.9217 - recall: 0.9036 - val_accuracy: 0.8861 - val_loss: 0.3782 - val_precision: 0.8984 - val_recall: 0.8844 - learning_rate: 1.5625e-06
Epoch 66/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 131s 853ms/step - accuracy: 0.9087 - loss: 0.3079 - precision: 0.9199 - recall: 0.8972 - val_accuracy: 0.8869 - val_loss: 0.3772 - val_precision: 0.9006 - val_recall: 0.8836 - learning_rate: 1.5625e-06
Epoch 67/100
153/153 ━━━━━━━━━━━━━━━━

153/153 ━━━━━━━━━━━━━━━━━━━━ 120s 783ms/step - accuracy: 0.9219 - loss: 0.2917 - precision: 0.9311 - recall: 0.9089 - val_accuracy: 0.8885 - val_loss: 0.3750 - val_precision: 0.9032 - val_recall: 0.8869 - learning_rate: 1.0000e-06
Epoch 72/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 120s 784ms/step - accuracy: 0.9168 - loss: 0.2920 - precision: 0.9272 - recall: 0.9103 - val_accuracy: 0.8877 - val_loss: 0.3775 - val_precision: 0.8999 - val_recall: 0.8844 - learning_rate: 1.0000e-06
Epoch 73/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 119s 780ms/step - accuracy: 0.9242 - loss: 0.2828 - precision: 0.9333 - recall: 0.9126 - val_accuracy: 0.8885 - val_loss: 0.3768 - val_precision: 0.9015 - val_recall: 0.8852 - learning_rate: 1.0000e-06
Epoch 74/100
153/153 ━━━━━━━━━━━━━━━━━━━━ 132s 859ms/step - accuracy: 0.9172 - loss: 0.3004 - precision: 0.9285 - recall: 0.9062 - val_accuracy: 0.8828 - val_loss: 0.3775 - val_precision: 0.8966 - val_recall: 0.8811 - learning_rate: 1.0000e-06
Epoch 75/100
153/153 ━━━━━━━━━━━━━━━━

Model saved as fake_product_detection_model.h5
Class map saved as class_map.json


In [1]:
# # Import libraries
# import os
# import cv2
# import numpy as np
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.utils import to_categorical
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.callbacks import EarlyStopping

# # Data paths
# base_path = r'D:\fake product\Data'
# train_path = os.path.join(base_path, 'train')
# test_path = os.path.join(base_path, 'test')

# # Correct preprocessing function
# def preprocess_images(dataset_path, image_size=(128, 128)):
#     images = []
#     labels = []
#     classes = sorted(os.listdir(dataset_path))
#     class_map = {label: idx for idx, label in enumerate(classes)}
    
#     for label in classes:
#         label_path = os.path.join(dataset_path, label)
#         for img_name in os.listdir(label_path):
#             img_path = os.path.join(label_path, img_name)
#             img = cv2.imread(img_path)
#             if img is not None:
#                 img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#                 img = cv2.resize(img, image_size)
#                 images.append(img)
#                 labels.append(class_map[label])
    
#     return np.array(images)/255.0, np.array(labels), class_map

# # Load data
# X_train, y_train, class_map = preprocess_images(train_path)
# X_test, y_test, _ = preprocess_images(test_path)

# # Verify shapes
# print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")

# # Split data
# X_train, X_val, y_train, y_val = train_test_split(
#     X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

# # Convert to PROPER categorical format
# num_classes = len(class_map)
# print(f"Number of classes: {num_classes}")  # Should be 2 (real/fake)

# # Fix the labels - ensure they're 1D before converting
# y_train = y_train.reshape(-1)  # Flatten if needed
# y_val = y_val.reshape(-1)
# y_test = y_test.reshape(-1)

# y_train_cat = to_categorical(y_train, num_classes=num_classes)
# y_val_cat = to_categorical(y_val, num_classes=num_classes)
# y_test_cat = to_categorical(y_test, num_classes=num_classes)

# # Verify fixed shapes
# print(f"\nCorrected shapes:")
# print(f"y_train_cat: {y_train_cat.shape} (should be N, {num_classes})")

# # Build model
# model = Sequential([
#     Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
#     MaxPooling2D(2,2),
#     Conv2D(64, (3,3), activation='relu'),
#     MaxPooling2D(2,2),
#     Conv2D(128, (3,3), activation='relu'),
#     MaxPooling2D(2,2),
#     Flatten(),
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(num_classes, activation='softmax')
# ])

# model.compile(optimizer=Adam(learning_rate=0.0001),
#              loss='categorical_crossentropy',
#              metrics=['accuracy'])

# # Train with augmentation
# train_datagen = ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True)

# history = model.fit(
#     train_datagen.flow(X_train, y_train_cat, batch_size=32),
#     epochs=50,
#     validation_data=(X_val, y_val_cat),
#     callbacks=[EarlyStopping(patience=5)],
#     verbose=1
# )

# # Evaluate
# test_loss, test_acc = model.evaluate(X_test, y_test_cat)
# print(f"Test Accuracy: {test_acc*100:.2f}%")



import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, BatchNormalization, 
                                    Dropout, Dense, Flatten, GlobalAveragePooling2D)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import (EarlyStopping, ReduceLROnPlateau, 
                                      ModelCheckpoint, TensorBoard)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

# Data paths
base_path = r'D:\fake product\Data'
train_path = os.path.join(base_path, 'train')
test_path = os.path.join(base_path, 'test')

# Enhanced preprocessing function
def preprocess_images(dataset_path, image_size=(224, 224)):
    images = []
    labels = []
    classes = sorted(os.listdir(dataset_path))
    class_map = {label: idx for idx, label in enumerate(classes)}
    
    for label in classes:
        label_path = os.path.join(dataset_path, label)
        for img_name in os.listdir(label_path):
            img_path = os.path.join(label_path, img_name)
            try:
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, image_size)
                    
                    # Apply CLAHE for contrast enhancement
                    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
                    l, a, b = cv2.split(lab)
                    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
                    cl = clahe.apply(l)
                    limg = cv2.merge((cl,a,b))
                    img = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)
                    
                    images.append(img)
                    labels.append(class_map[label])
            except Exception as e:
                print(f"Error processing {img_path}: {str(e)}")
    
    return np.array(images)/255.0, np.array(labels), class_map

# Load data
print("Loading and preprocessing data...")
X_train, y_train, class_map = preprocess_images(train_path)
X_test, y_test, _ = preprocess_images(test_path)

# Verify shapes
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")

# Split data
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

# Convert to categorical
num_classes = len(class_map)
print(f"Number of classes: {num_classes}")

y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_val_cat = to_categorical(y_val, num_classes=num_classes)
y_test_cat = to_categorical(y_test, num_classes=num_classes)

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Transfer Learning with EfficientNet
def create_transfer_model(input_shape=(224, 224, 3), num_classes=2):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # Freeze initial layers
    for layer in base_model.layers[:100]:
        layer.trainable = False
    
    # Add custom head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.3)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model

# Create model
model = create_transfer_model(input_shape=X_train.shape[1:], num_classes=num_classes)

# Custom CNN Model (alternative)
def create_cnn_model(input_shape=(224, 224, 3), num_classes=2):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.25),
        
        Conv2D(64, (3,3), activation='relu', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.25),
        
        Conv2D(128, (3,3), activation='relu', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.25),
        
        Conv2D(256, (3,3), activation='relu', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.25),
        
        GlobalAveragePooling2D(),
        Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Uncomment to use CNN instead of transfer learning
# model = create_cnn_model(input_shape=X_train.shape[1:], num_classes=num_classes)

# Compile model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
             loss='categorical_crossentropy',
             metrics=['accuracy', 
                    tf.keras.metrics.Precision(name='precision'),
                    tf.keras.metrics.Recall(name='recall')])

# Callbacks
callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True, monitor='val_loss'),
    ReduceLROnPlateau(factor=0.5, patience=3, min_lr=1e-6),
    ModelCheckpoint('best_model.h5', save_best_only=True),
    TensorBoard(log_dir='./logs')
]

# Train model
print("Training model...")
history = model.fit(
    train_datagen.flow(X_train, y_train_cat, batch_size=32),
    epochs=100,
    validation_data=(X_val, y_val_cat),
    callbacks=callbacks,
    verbose=1
)

# Evaluate
print("Evaluating model...")
test_loss, test_acc, test_precision, test_recall = model.evaluate(X_test, y_test_cat)
print(f"\nTest Accuracy: {test_acc*100:.2f}%")
print(f"Test Precision: {test_precision*100:.2f}%")
print(f"Test Recall: {test_recall*100:.2f}%")

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.legend()
plt.tight_layout()
plt.savefig('training_history.png')
plt.show()

# Save model
model.save('fake_product_detection_model.h5')
print("Model saved as fake_product_detection_model.h5")

# Generate class map file
import json
with open('class_map.json', 'w') as f:
    json.dump(class_map, f)
print("Class map saved as class_map.json")